In [2]:
from hashlib import md5
# from sklearn import tree
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from pathlib import Path
from datetime import datetime

In [3]:
# def MSE(y_true, y_pred):
#     """
#     Returns the Mean Squared Error
#     """
#     return ((y_true - y_pred) ** 2).mean()



In [4]:
def get_data():
    """
    Returns the data from the xlsx file
    """
    file_name = 'yeast.csv' 
    df = pd.read_csv(file_name, index_col=0, header=None)
    # print(df.head()) # print the first 5 rows
    return df

md = get_data()
md.dropna(inplace = True)
print(md.shape)
md.replace('?', 0, inplace = True)

print(md)

(1484, 9)
               1     2     3     4    5    6     7     8    9
0                                                            
ADT1_YEAST  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22  MIT
ADT2_YEAST  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22  MIT
ADT3_YEAST  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22  MIT
AAR2_YEAST  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22  NUC
AATM_YEAST  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22  MIT
...          ...   ...   ...   ...  ...  ...   ...   ...  ...
YUR1_YEAST  0.81  0.62  0.43  0.17  0.5  0.0  0.53  0.22  ME2
ZIP1_YEAST  0.47  0.43  0.61  0.40  0.5  0.0  0.48  0.47  NUC
ZNRP_YEAST  0.67  0.57  0.36  0.19  0.5  0.0  0.56  0.22  ME2
ZUO1_YEAST  0.43  0.40  0.60  0.16  0.5  0.0  0.53  0.39  NUC
G6PD_YEAST  0.65  0.54  0.54  0.13  0.5  0.0  0.53  0.22  CYT

[1484 rows x 9 columns]


In [5]:
X = md.iloc[:, 0:8].values.reshape(-1, 8)


print(X)
print(X.shape)
# add 1 to the first column
X = np.insert(X, 0, 1, axis=1)
print('X:', X)
print(X.shape)

Y = md.iloc[:, 8:9].values.reshape(-1, 1)


[[0.58 0.61 0.47 ... 0.   0.48 0.22]
 [0.43 0.67 0.48 ... 0.   0.53 0.22]
 [0.64 0.62 0.49 ... 0.   0.53 0.22]
 ...
 [0.67 0.57 0.36 ... 0.   0.56 0.22]
 [0.43 0.4  0.6  ... 0.   0.53 0.39]
 [0.65 0.54 0.54 ... 0.   0.53 0.22]]
(1484, 8)
X: [[1.   0.58 0.61 ... 0.   0.48 0.22]
 [1.   0.43 0.67 ... 0.   0.53 0.22]
 [1.   0.64 0.62 ... 0.   0.53 0.22]
 ...
 [1.   0.67 0.57 ... 0.   0.56 0.22]
 [1.   0.43 0.4  ... 0.   0.53 0.39]
 [1.   0.65 0.54 ... 0.   0.53 0.22]]
(1484, 9)


In stochastic (or "on-line") gradient descent, the true gradient of
*Q*(*w*) is approximated by a gradient at a single sample:

  
*w* := *w* − *η*∇*Q*<sub>*i*</sub>(*w*).

As the algorithm sweeps through the training set, it performs the above
update for each training sample. Several passes can be made over the
training set until the algorithm converges. If this is done, the data
can be shuffled for each pass to prevent cycles. Typical implementations
may use an [adaptive learning rate] so that the algorithm converges.[1]

In pseudocode, stochastic gradient descent can be presented as :

<div style="margin-left: 35px; width: 600px">
-   Choose an initial vector of parameters *w* and learning rate *η*.
-   Repeat until an approximate minimum is obtained:
    -   Randomly shuffle samples in the training set.
    -   For *i* = 1, 2, ..., *n*, do:
        -   *w* := *w* − *η*∇*Q*<sub>*i*</sub>(*w*).

[1]

  [adaptive learning rate]: adaptive_learning_rate "wikilink"

In [6]:
def p(i):
    return 1 / (1 + np.exp(-i))

In [12]:
def pi(x):
    return np.exp(np.dot(x, beta[0])) / (1 + np.exp(np.dot(x, beta[0])))

In [7]:
outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

y = np.zeros((len(Y), 10))
print(y.shape)

for i in range(len(Y)):
    for j in range(10):
        if outputs[j] == Y[i]:
            y[i][j] = 1
        else:
            y[i][j] = 0

print('results', Y[0] == outputs[2])
print(y.shape)
# for i in range(10):
#     print(outputs[i], ': ', y[i])

(1484, 10)
results [ True]
(1484, 10)


In [16]:
eta = 0.01
beta = np.zeros((10, 9))

gradient = np.zeros(9)

# gradient[0] = 9999

# stochastic gradient descent
n = 0
while (n < 1000):
#while (np.linalg.norm(gradient) > 0.01):
    # gradient = np.zeros(9)

    # take one sample randomly
    i = np.random.randint(0, X.shape[0])
    print(i)
    
    # calculate the gradient
    for j in range(9):
        gradient[j] = (p(np.dot(X[i], beta[0])) - y[i][0]) * X[i][j]

    print('grad', gradient)

    # update beta
    beta[0] = beta[0] - eta * gradient
    
    print(beta[0])
    n = n + 1

564
grad [0.5   0.23  0.285 0.26  0.14  0.25  0.    0.26  0.11 ]
[-0.005   -0.0023  -0.00285 -0.0026  -0.0014  -0.0025   0.      -0.0026
 -0.0011 ]
1073
grad [-0.50306634 -0.24147184 -0.25656383 -0.25153317 -0.12073592 -0.25153317
 -0.         -0.27165582 -0.19116521]
[ 3.06633656e-05  1.14718415e-04 -2.84361684e-04 -8.46683172e-05
 -1.92640792e-04  1.53316828e-05  0.00000000e+00  1.16558217e-04
  8.11652079e-04]
1106
grad [0.50007269 0.23003344 0.24003489 0.19002762 0.11001599 0.25003634
 0.         0.19502835 0.22503271]
[-0.00497006 -0.00218562 -0.00268471 -0.00198494 -0.0012928  -0.00248503
  0.         -0.00183373 -0.00143868]
150
grad [0.49719752 0.28837456 0.24859876 0.24859876 0.0596637  0.24859876
 0.         0.23865481 0.12429938]
[-0.00994204 -0.00506936 -0.0051707  -0.00447093 -0.00188944 -0.00497102
  0.         -0.00422027 -0.00268167]
36
grad [0.49381038 0.35554348 0.30122433 0.22221467 0.14814312 0.24690519
 0.         0.26665761 0.1234526 ]
[-0.01488014 -0.0086248  -0.

In [15]:
for i in range(len(X)):
    print(pi(X[i]), y[i][0])

0.30887934300911823 0.0
0.3052406129553081 0.0
0.303354604415287 0.0
0.3142327472050512 0.0
0.3079423476044194 0.0
0.3170448904901484 1.0
0.2940608663491589 0.0
0.30989461650743805 0.0
0.30393180690901317 0.0
0.3194618478044173 1.0
0.3190596042034197 0.0
0.31988735524067535 0.0
0.31360178665380084 1.0
0.3034281829761785 0.0
0.30695017202937486 0.0
0.3220463890122863 1.0
0.3187181089356526 1.0
0.3134966663239932 0.0
0.3212774343327875 0.0
0.30244040805042566 0.0
0.32276180442458413 1.0
0.3191867370686847 1.0
0.29541099079005084 1.0
0.3165289566405163 1.0
0.31355571122109194 1.0
0.2996386027767346 0.0
0.3083526988525577 1.0
0.311723601156026 1.0
0.2980647765123448 0.0
0.2728594634023356 0.0
0.31020723621222046 0.0
0.28769438256780244 0.0
0.30638507091831596 1.0
0.30936171415136743 0.0
0.2913303843739931 0.0
0.29296929353359297 0.0
0.29402162194529324 0.0
0.3175796524530048 1.0
0.3002982864602023 0.0
0.3028545447098473 0.0
0.28887817785461845 0.0
0.27840752115655587 0.0
0.2938050370364349